In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Import pandas
import pandas as pd
import numpy as np

In [ ]:
#load dataset into dataframe
df = pd.read_csv('/content/drive/MyDrive/Concepts and Technologies of AI./5CS037 - Cohort 11+12 - 2024 - Materials/Week -4-  Introduction to Learning Theory for Machine Learning/Dataset/diabetes.csv')

Problem 1

In [ ]:
#display first 10 rows
df.head(10)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
5,5,116,74,0,0,25.6,0.201,30,0
6,3,78,50,32,88,31.0,0.248,26,1
7,10,115,0,0,0,35.3,0.134,29,0
8,2,197,70,45,543,30.5,0.158,53,1
9,8,125,96,0,0,0.0,0.232,54,1


In [ ]:
#check for missing value
df.isnull().sum()

,0
Pregnancies,0
Glucose,0
BloodPressure,0
SkinThickness,0
Insulin,0
BMI,0
DiabetesPedigreeFunction,0
Age,0
Outcome,0


In [ ]:
#check datatypes and column
df.dtypes

,0
Pregnancies,int64
Glucose,int64
BloodPressure,int64
SkinThickness,int64
Insulin,int64
BMI,float64
DiabetesPedigreeFunction,float64
Age,int64
Outcome,int64


In [ ]:
#target variable
Y = df["Outcome"]

#feature matrix
X = df.drop(columns=["Outcome"])

In [ ]:
# Set a random seed for reproducibility
np.random.seed(42)

# Shuffle the indices
indices = np.arange(len(df))
np.random.shuffle(indices)

# Split into train and test
split_index = int(0.7 * len(df))
train_indices = indices[:split_index]
test_indices = indices[split_index:]

# Create train and test sets
X_train, X_test = X.iloc[train_indices], X.iloc[test_indices]
y_train, y_test = y.iloc[train_indices], y.iloc[test_indices]


In [ ]:
def euclidean_distance(point1, point2):
    return np.sqrt(np.sum((point1 - point2) ** 2))

In [ ]:
def predict_single(query, X_train, y_train, k):
    distances = []

    # Compute distances to all points in the training set
    for i in range(len(X_train)):
        dist = euclidean_distance(query, X_train.iloc[i])
        distances.append((dist, y_train.iloc[i]))

    # Sort by distance and get the k nearest neighbors
    distances.sort(key=lambda x: x[0])
    k_neighbors = distances[:k]

    # Get the majority class among the k neighbors
    classes = [neighbor[1] for neighbor in k_neighbors]
    prediction = max(set(classes), key=classes.count)

    return prediction

In [ ]:
def predict_all(X_test, X_train, y_train, k):
    predictions = []
    for i in range(len(X_test)):
        pred = predict_single(X_test.iloc[i], X_train, y_train, k)
        predictions.append(pred)
    return predictions


In [ ]:
def accuracy_score(y_true, y_pred):
    correct = sum(y_true == y_pred)
    return correct / len(y_true)


In [ ]:
# Set the value of k
k = 5

# Predict for all test samples
y_pred = predict_all(X_test, X_train, y_train, k)

# Evaluate accuracy
acc = accuracy_score(y_test.values, y_pred)
print(f"Accuracy: {acc:.2f}")


Accuracy: 0.71


Problem 2

In [ ]:
def scale_features(X):
    return (X - X.mean()) / X.std()

# Scale the feature matrix
X_scaled = scale_features(X)


In [ ]:
# Use the same train-test indices
X_train_scaled, X_test_scaled = X_scaled.iloc[train_indices], X_scaled.iloc[test_indices]


In [ ]:
# Predict for all test samples using scaled data
y_pred_scaled = predict_all(X_test_scaled, X_train_scaled, y_train, k)

# Evaluate accuracy for the scaled data
acc_scaled = accuracy_score(y_test.values, y_pred_scaled)
print(f"Accuracy with Scaled Data: {acc_scaled:.2f}")


Accuracy with Scaled Data: 0.74


Observations:
Impact of Scaling on KNN:
Scaling typically improves KNN performance because the algorithm depends on the Euclidean distance, which is sensitive to the magnitude of the features. Features with larger ranges can dominate the distance computation, leading to biased results.

Accuracy Difference:
If accuracy improves after scaling, it shows that the feature magnitudes were initially imbalanced, and scaling helped to equalize their contributions to distance calculations.
If accuracy doesn't change significantly, the features might have already been on similar scales.

Reason for Improved Performance:
Without scaling, features like BMI (float values) and Pregnancies (integers) might have disproportionate influence during distance calculation.
Scaling standardizes all features to have a mean of 0 and a standard deviation of 1, ensuring each feature contributes equally to the distance metric.

Final Discussion:
Key Insight: Scaling the data is crucial for algorithms like KNN that rely on distance metrics. It ensures fair representation of all features and reduces bias.

Recommendation: Always scale features when using KNN, especially if features are measured on different scales.

Problem 3

In [ ]:
import time

# Function to evaluate kNN with different values of k
def evaluate_knn(X_train, X_test, y_train, y_test, k_values):
    results = {
        "k": [],
        "accuracy_original": [],
        "time_original": [],
        "accuracy_scaled": [],
        "time_scaled": []
    }

    for k in k_values:
        # Original Data
        start_time = time.time()
        y_pred_original = predict_all(X_test, X_train, y_train, k)
        time_original = time.time() - start_time
        acc_original = accuracy_score(y_test.values, y_pred_original)

        # Scaled Data
        start_time = time.time()
        y_pred_scaled = predict_all(X_test_scaled, X_train_scaled, y_train, k)
        time_scaled = time.time() - start_time
        acc_scaled = accuracy_score(y_test.values, y_pred_scaled)

        # Record Results
        results["k"].append(k)
        results["accuracy_original"].append(acc_original)
        results["time_original"].append(time_original)
        results["accuracy_scaled"].append(acc_scaled)
        results["time_scaled"].append(time_scaled)

    return results

# Define k values
k_values = range(1, 16)

# Run evaluation
results = evaluate_knn(X_train, X_test, y_train, y_test, k_values)


In [ ]:
import matplotlib.pyplot as plt

# Plot k vs. Accuracy
plt.figure(figsize=(12, 6))
plt.plot(results["k"], results["accuracy_original"], label="Original Data", marker="o")
plt.plot(results["k"], results["accuracy_scaled"], label="Scaled Data", marker="o")
plt.title("k vs. Accuracy")
plt.xlabel("k (Number of Neighbors)")
plt.ylabel("Accuracy")
plt.legend()
plt.grid(True)
plt.show()

# Plot k vs. Time Taken
plt.figure(figsize=(12, 6))
plt.plot(results["k"], results["time_original"], label="Original Data", marker="o")
plt.plot(results["k"], results["time_scaled"], label="Scaled Data", marker="o")
plt.title("k vs. Time Taken")
plt.xlabel("k (Number of Neighbors)")
plt.ylabel("Time Taken (seconds)")
plt.legend()
plt.grid(True)
plt.show()


Impact of k on Accuracy:
For both datasets, the accuracy initially increases as k grows because it averages out noise.
Beyond a certain k, accuracy may drop slightly as the classifier becomes too general, potentially misclassifying points in minority classes.

Impact of k on Computational Cost:
The time taken for prediction increases with k, as more neighbors must be considered for each query point.

Comparison Between Original and Scaled Data:
The scaled dataset generally achieves higher accuracy because all features contribute equally to the distance computation.
The computational cost is similar for both datasets since scaling only affects the distance calculation, not the number of neighbors.

Optimal k:
The optimal k is typically the value that achieves the highest accuracy with minimal drop-off. This can be identified by inspecting the accuracy plot.
For balanced datasets, a moderate k (e.g., 5 or 7) often works well.

Problem 4

Computational Cost:
Reason: KNN requires calculating distances between the query point and every point in the dataset, making it computationally expensive for large datasets.
Impact: For a dataset with
n samples and d dimensions, the time complexity for each query is (𝑛⋅𝑑) O(n⋅d), which scales poorly.

Memory Requirements:
Reason: KNN needs to store the entire dataset in memory to perform predictions.
Impact: For large datasets, this can result in significant memory consumption.

Curse of Dimensionality:
Reason: In high-dimensional spaces, data points tend to become equidistant due to the sparsity of data, making it difficult to distinguish between neighbors and non-neighbors.

Impact: This reduces the effectiveness of the Euclidean distance metric, leading to poor classification performance.

Sensitivity to Irrelevant Features:
Reason: Irrelevant or noisy features increase the distance computation complexity and can overshadow relevant features.
Impact: This leads to reduced accuracy and makes the model harder to interpret.
Strategies to Improve the Efficiency of KNN

Approximate Nearest Neighbors (ANN)
Method: Use algorithms like KD-Trees, Ball Trees, or Locality-Sensitive Hashing (LSH) to approximate nearest neighbors rather than computing exact distances.
Benefit: Significantly reduces computation time, especially for large datasets.
Trade-off: Accuracy may slightly decrease, but the computational gain is often worth it.

Dimensionality Reduction Techniques:
Principal Component Analysis (PCA): Reduces dimensionality by projecting data onto a lower-dimensional space while preserving maximum variance.
t-SNE or UMAP: Useful for visualizing and reducing high-dimensional data.
Benefit: Reduces the impact of the curse of dimensionality and computational cost.
Trade-off: May result in information loss if too many dimensions are removed.

Feature Selection
Method: Identify and retain only the most relevant features using techniques like mutual information, correlation analysis, or recursive feature elimination.
Benefit: Reduces irrelevant noise and computational overhead.
Trade-off: Requires preprocessing and careful feature engineering.

Distance Metric Optimization
Method: Experiment with other distance metrics (e.g., Manhattan distance, Minkowski distance) or custom metrics better suited to the dataset.
Benefit: Improves accuracy by tailoring the metric to the problem domain.
Trade-off: Might require domain-specific knowledge.

Precomputing Distances
Method: For datasets with many repeated queries, precompute and cache distances using data structures like distance matrices.
Benefit: Reduces repeated computations.
Trade-off: Increases memory usage.

Sampling or Data Partitioning
Method: Use a subset of the training data by sampling or divide the data into partitions and search locally.
Benefit: Reduces computational time while maintaining reasonable accuracy.
Trade-off: Risk of missing relevant neighbors if the sample or partition is not representative.

Parallelization
Method: Distribute distance computations across multiple processors or GPUs.
Benefit: Speeds up computations for large datasets.
Trade-off: Requires specialized hardware and software.

Discussion
KNN Strengths: Simplicity, effectiveness for small datasets, and interpretability.
KNN Weaknesses: Scalability and sensitivity to irrelevant features in high dimensions.

Best Practices:
Scale and preprocess your data to minimize noise.
Consider approximate methods or dimensionality reduction for large datasets.
Use KNN as a baseline or benchmark model before transitioning to more sophisticated models for production.